# <u>Reset Test Environment</u>

This notebook will use C#, SQL and PowerShell

Use this notebook to reset my test environment for the "Intro to SQL Tools" session.

**NOTE:** This works best if no tools are open and this tool has everything closed as well.

Pull down the .NET interactive packages for connecting to SQL Server before starting:

In [2]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"
/*#r "nuget: Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.22405.1"*/

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.22405.1

## <u>Copy SQL Server Backup File:</u>

- Make sure the backup is in the shared directory

In [ ]:
$session = 'DataIntegrity'
$backupfile = 'C:\GitRepo\SQL_Presentations\Superhero Demo DB Setup\Superheroes_SQL2019_' + $Session + '_Initial.BAK'

Copy-Item $backupfile -Destination 'L:\Backups\Superheroes'

## <u> SQL Server clean up:</u>
* Drop the SuperheroesPublish database if it exists
* Restore the Superheroes back to the original
* Drop the notebook job if it exists

In [ ]:
#!connect mssql --kernel-name mydatabase "Persist Security Info=False; Integrated Security=true; Initial Catalog=master; Server=192.168.1.152"

In [ ]:
#!sql-mydatabase

DECLARE @session varchar(30), @backupfile nvarchar(300), @today DATE = getdate()

SELECT @Session = 'DataIntegrity'
SELECT @backupfile = '\\192.168.1.150\Lab Share\backups\Superheroes\Superheroes_SQL2019_' + @Session + '_Initial.BAK'
SELECT @backupfile

DECLARE @backuplogname nvarchar(50) = 'Superheroes_LogBackup' + convert(varchar(10), @today) + '.BAK'

ALTER DATABASE Superheroes SET SINGLE_USER WITH ROLLBACK IMMEDIATE;


RESTORE DATABASE [Superheroes] 
FROM  DISK = @backupfile WITH  FILE = 1, 
    MOVE N'Superheroes' TO N'D:\MSSQL16.MSSQLSERVER\MSSQL\DATA\Superheroes_Data.mdf',  
    MOVE N'Superheroes_log' TO N'L:\MSSQL16.MSSQLSERVER\MSSQL\Log\Superheroes_Log.ldf',  
    NOUNLOAD,  REPLACE,  STATS = 5;

ALTER DATABASE Superheroes SET MULTI_USER;


## <u>Directory Clean Up:</u>

Clean up directories

- Clear the output files directory
- Clear the Database Project directory

In [ ]:
$outputfileslocation = "C:\GitRepo\Intro to SQL Tools\Output Files"

Get-ChildItem -Path $outputfileslocation | WHERE {$_.Name -like 'Superheroes*'} | ForEach-Object {
    $name = $_.FullName

    write-host "Removing item $name ..."
    Remove-Item $name -Recurse -Force
}

In [ ]:
$dbprojlocation = "C:\GitRepo\Intro to SQL Tools\Superheroes DBProj"

Get-ChildItem -Path $dbprojlocation | WHERE {$_.Name -like 'Superheroes*'} | ForEach-Object {
    $name = $_.FullName

    write-host "Removing item $name ..."
    Remove-Item $name -Recurse -Force
}